#### matplotlib 提供了一个非常有用的注解工具annotations，可以在数据图形上添加文本注解 ，注解通常用于解释数据的内容

In [383]:
%matplotlib inline

In [384]:
import matplotlib.pyplot as plt

#### 使用文本注解绘制树节点

In [385]:
#创建字典的另一种方法，字典的内置函数dict（）创建
#决策点的属性， boxstyle为文本框的类型，sawtooth是锯齿形，fc是文本框内的颜色  
decisionNode = dict(boxstyle = "sawtooth", fc = "0.8")

In [386]:
leafNode = dict(boxstyle = "round4", fc =  "0.8")#叶子节点的属性

In [387]:
arrow_args = dict(arrowstyle = "<-")#箭头的属性

In [388]:
def plotNode(nodeTxt, centerPt, parentPt, nodeType):
#nodeTxt为要显示的文本，xy：（在完整的树结构图中为父节点坐标即箭头的相反方向，也即参数parentPt），
#xytext：注释文本的位置坐标（在后续完整的树结构图中为节点的中心点，也即箭头所在的点centerPt）， nodeType为前面定义的决策点或叶子节点
#xycoords 和 textcoords 是字符串，指示 xy 和 xytext 的坐标关系(坐标xy与xytext的说明)：若textcoords=None，则
#默认textcoords与xycoords相同，若都未设置，默认为data
#va/ha设置节点框中文字的位置，va为纵向取值为(u'top', u'bottom', u'center', u'baseline')，ha为横向取值为(u'center', u'right', u'left'
    createPlot.ax1.annotate(nodeTxt, xy = parentPt, xycoords = 'axes fraction', xytext = centerPt, textcoords = 'axes fraction',\
                           va = "center", ha = "center", bbox = nodeType, arrowprops = arrow_args)

In [389]:
def createPlot():
    #创建一个画布，背景为白色
    fig = plt.figure(1, facecolor = 'white')
    #画布清空
    fig.clf()
    #ax1是函数createPlot的一个属性，这个可以在函数里面定义也可以在函数定义后加入也可以
    #frameon表示是否绘制坐标轴矩形    
    createPlot.ax1 = plt.subplot(111, frameon = False)
    plotNode('a decision node', (0.5, 0.1), (0.1, 0.5), decisionNode)
    plotNode('a leaf node',(0.8, 0.1), (0.3, 0.8), leafNode)
    plt.show()

#### 获取叶节点的数目和树的层数 （在该例中如果键值是类标签（yes或no），则该节点是叶子节点；若值是另一个字典，则该节点是判断节点）

In [390]:
#叶节点的数目
def getNumLeafs(myTree):
    #定义叶子节点数目变量
    numLeafs = 0
    #获得myTree的第一个键，即第一个特征
    firstStr = myTree.keys()[0]
    # 根据键得到对应的键值，即根据第一个特征分类的结果 
    secondDict = myTree[firstStr]
    #遍历secondDict字典的键
    for key in secondDict.keys():
        #若相应的键的键值又为一个字典，即判断节点，则进行递归，直到为叶子节点
        if type(secondDict[key]).__name__=='dict':
            numLeafs += getNumLeafs(secondDict[key])
        #若相应的键的键值不是字典，是类标签，则为叶子节点，numLeafs加1
        else:   numLeafs +=1
    #返回叶子节点的数目
    return numLeafs

In [391]:
#获取树的层数(计算遍历过程中遇到判断节点的个数，即层数)
def getTreeDepth(myTree):
    #定义关于树的层数的变量
    maxDepth = 0
    #获得myTree的第一个键，即第一个特征
    firstStr = myTree.keys()[0]
     # 根据键得到对应的键值，即根据第一个特征分类的结果 
    secondDict = myTree[firstStr]
   # print secondDict
    #遍历secondDict字典的键
    for key in secondDict.keys():
        #print key
        #若相应的键的键值又为一个字典，即判断节点，则进行递归，直到为叶子节点终止，一旦到达叶子节点，
        #则从递归调用中返回，并将计算树的深度加1
        if type(secondDict[key]).__name__=='dict':
            thisDepth = 1 + getTreeDepth(secondDict[key])
        #若为叶子节点，则当前树的深度设为1
        else:   thisDepth = 1
        # 如果当前树的深度大于数最大深度  
        if thisDepth > maxDepth: maxDepth = thisDepth
    #返回树的最大深度
    return maxDepth

In [392]:
#绘制中间文本 ,即每一个判断节点划分结果（0和1）
def plotMidText(cntrPt, parentPt, txtString):
    ## 求中间点的横坐标
    xMid = (parentPt[0]-cntrPt[0])/2.0 + cntrPt[0]
    # 求中间点的纵坐标  
    yMid = (parentPt[1]-cntrPt[1])/2.0 + cntrPt[1]
    #绘制0或1文本
    createPlot.ax1.text(xMid, yMid, txtString, va="center", ha="center", rotation=30)


#### 变量plotTree.totalW存储树的宽度，plotTree.totalD存储树的深度（用这两个变量计算树节点的摆放位置，这样可以将树绘制在水平方向和垂直方向的中心位置）；

#### plotTree.xoff和plotTree.yoff追踪已经绘制的节点位置，以及放置下一个节点的恰当位置。

### 树的宽度用于计算放置判断节点的位置，主要的计算原则是将它放在它的所有叶子节点的中间，而不仅仅是它的子节点的中间。

### 通过计算树包含的所有叶子节点数，划分图形的宽度，从而计算得到当前节点的中心位置，也就是说，我们按照叶子节点的数目将X轴化分为若干部分 。每一部分的中点的x坐标即为叶子节点的横坐标（ plotTree.xOff = plotTree.xOff + 1.0/plotTree.totalW）；判断节点的x坐标是它所有叶子节点的中间（例如本例中no surfacing判断节点有3个叶子节点，flippers判断节点有2个叶子节点）

In [393]:
def plotTree(myTree, parentPt, nodeTxt):
    #计算树叶子节点（宽度）
    numLeafs = getNumLeafs(myTree) 
    #计算树的判断节点（高度）
    depth = getTreeDepth(myTree)
    #获得myTree的第一个键，即第一个特征
    firstStr = myTree.keys()[0] 
    #cntrPt
    #其中，变量plotTree.xOff即为最近绘制的一个叶子节点的x轴坐标，
    #在确定当前节点位置时每次只需确定当前节点有几个叶子节点，因此其叶子节点所占的
    #总距离就确定了即为： float(numLeafs)/plotTree.totalW，因此当前节点的位置即为其所有叶子节点
    #所占距离的中间即一半为： float(numLeafs)/2.0/plotTree.totalW，但是由于开始plotTree.xOff赋值
    #并非从0开始，而是左移了半个表格，因此还需加上半个表格距离即为： 1/2/plotTree.totalW，
    #则加起来便为： (1.0 + float(numLeafs))/2.0/plotTree.totalW，因此偏移量确定，则x轴的位置变为： 
    # plotTree.xOff + (1.0 + float(numLeafs))/2.0/plotTree.totalW（是计算判断节点x轴的坐标）  
    cntrPt = (plotTree.xOff + (1.0 + float(numLeafs))/2.0/plotTree.totalW, plotTree.yOff)
   # print numLeafs
   # print cntrPt
    #绘制中间文本（cntrPt为箭头所指节点中心的坐标,parentPt为父节点中心坐标也即箭头的反方向）
    #在创建第个决策树节点（no surfacing）时,由于该节点的parentPt和指向该节点的cntrPt坐标相等，是一个点都为（0.5，1）,打印cntrPt
    #所以对于该决策树节点的绘制，只有一个节点，没有箭头和中间文本（nodeTxt为空（‘’））
    plotMidText(cntrPt, parentPt, nodeTxt)
    #绘制第一个决策树节点（判断节点）
    plotNode(firstStr, cntrPt, parentPt, decisionNode)
    #根据键firstStr取出对应的键值
    secondDict = myTree[firstStr]
    #因为进入了下一层，所以y的坐标要变
    plotTree.yOff = plotTree.yOff - 1.0/plotTree.totalD
    #遍历 secondDict字典的键
    for key in secondDict.keys():
        #如果secondDict[key]为一棵子决策树，即字典 
        if type(secondDict[key]).__name__=='dict':  
            # 递归的绘制
            plotTree(secondDict[key],cntrPt,str(key))        
        else:
            #该公式是计算叶子节点x轴坐标的公式
            plotTree.xOff = plotTree.xOff + 1.0/plotTree.totalW
            #绘制叶子节点
            plotNode(secondDict[key], (plotTree.xOff, plotTree.yOff), cntrPt, leafNode)
            #绘制叶子节点的中间文本
            plotMidText((plotTree.xOff, plotTree.yOff), cntrPt, str(key))
    #将纵坐标上升一层
    plotTree.yOff = plotTree.yOff + 1.0/plotTree.totalD
   # plotNode(secondDict[key], (plotTree.xOff, plotTree.yOff), cntrPt, leafNode)

In [394]:
def createPlot(inTree):
    #定义一块画布(画布是自己的理解) ，背景为白色
    fig = plt.figure(1, facecolor='white')
    # 清空画布 
    fig.clf()
    #xticks和yticks: 为x,y轴的主刻度和次刻度设置颜色、大小、方向，以及标签大小。定义横纵坐标轴，无内容  
    axprops = dict(xticks=[], yticks=[])
    #绘制图像，无边框，无坐标轴 
    createPlot.ax1 = plt.subplot(111,frameon=False, **axprops)  
    #存储树的宽度（所有叶子节点）
    plotTree.totalW = float(getNumLeafs(inTree))
    #存储树的深度（判断节点的数目）
    plotTree.totalD = float(getTreeDepth(inTree))
   # print plotTree.totalD
    ## 决策树起始横坐标  1/plotTree.totalW是按照叶子节点的数目将x轴划分为若干个部分的每一部分的长度，
    #乘以-0.5即意为x开始位置为第一个表格左边的半个表格距离位置
    plotTree.xOff = -0.5/plotTree.totalW;
    # 决策树的起始纵坐标
    plotTree.yOff = 1.0;
    # 绘制决策树 
    plotTree(inTree, (0.5,1.0), '')
    #显示
    plt.show()

#### 预定义的树，主要用来测试

In [395]:
def retrieveTree(i):
    listOfTrees =[{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}},
                  {'no surfacing': {0: 'no', 1: {'flippers': {0: {'head': {0: 'no', 1: 'yes'}}, 1: 'no'}}}}
                  ]
    return listOfTrees[i]
